This notebook is to introduce you to curve fitting in python. The majority of this code is to introduce you to the various components of the fitting code. We use the plots/fits of Lecture 5 to do this. For lab, you can copy the code in the cell labeled #SUMMARY FOR LAB, and edit the various components. You won't need the other cells for lab.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#This cell makes the data for the example plots shown in lecture 5.
#The "data" in that lecture was generated by this code--it was not real.

npts = 15; #Number of data points to generate
xmin = 0; #min value of data range
xmax = 1.7; # max value of data range
x = np.linspace(xmin, xmax, npts); #this is the independent variable

#Let's make a "data curve" that looks like y = x + x^2 + x^3, and then we'll add a little noise.
y   = x+ x**2 + x**3 ; #this is a smooth curve, no noise.
y = y + np.random.normal(0, 0.4, npts) # now we add a little noise. The details of how we add noise aren't super important. This is to mimick experimental uncertainty.

dy =0.1+np.abs(np.random.normal(0, 0.4, npts)); #this variable will represent the uncertainty in each data point. Again here the details aren't important.

In [ ]:
#This cell makes a plot of the data. This code should look somewhat familiar from our python plotting assignment.
plt.rcParams.update({'font.size': 15})
plt.errorbar(x, y, yerr = dy, xerr = None, fmt = 'k.');
plt.xlabel('Indep Variable');
plt.ylabel('Dep Variable');

In [ ]:
#Let's try fitting this data with a line. We'll use the function in numpy called "polyfit" to do a polynomial fit (of degree 2).
#Polyfit can do any order polynomial. Here we choose a polynomial of order 1, or a line.
#Polyfit makes a chi-squared, and minimizes it. Then it spits out the coefficients of that polynomial in the resulting numpy array.

polynomial_degree = 1
p = np.polyfit(x, y, polynomial_degree, w = 1/dy ) ;  # this line performs the fit. p contains the coefficients of the polynomial. The syntax is not super obvious, I would say. The piece "w = " specifies the "weight" to apply to each of the residuals. As in lecture, the weight should be set to 1/(uncertainty).
f = np.poly1d(p) ; #This is a super handy line. It converts the list of polynomial coefficients (p), into the corresponding polynomial function (f).

#Here's how we can use f:

plt.errorbar(x, y, yerr = dy, xerr = None, fmt = 'k.'); #First make the plot from before.

plt.plot(x, f(x), 'r-'); # Now add to that plot the fit line. Notice we can just use f like a fucntion! It's our fit function.

plt.xlabel('Indep Variable');
plt.ylabel('Dep Variable');
plt.legend(['Fit', 'Data']);

In [ ]:
#Aside: What if we didn't include the uncertainties?
polynomial_degree = 1
p_no_unc = np.polyfit(x, y, polynomial_degree) ;  # same as before, except we've left out 'w', which tells the fitter what the uncertainties are. When we leave it out, it treats all points as if they have the same uncertainty.
f_no_unc = np.poly1d(p_no_unc) ; #same as before
plt.errorbar(x, y, yerr = dy, xerr = None, fmt = 'k.'); #same as before
plt.plot(x, f_no_unc(x), 'r-'); #same as before
plt.xlabel('Indep Variable');
plt.ylabel('Dep Variable');
plt.legend(['Fit', 'Data']);
#Look at the plot closely: Note that this changes the fit! the fit doesn't try as hard to hit the points with very small uncertainty.

In [ ]:
#What if we try to fit with a parabola? From here on out we will always include uncertainties in our fits (meaning, we will make a proper chi-squared.)
polynomial_degree = 2
p_parabola = np.polyfit(x, y, polynomial_degree, w = 1/dy) ;  # now we include uncertainties again.
f_parabola = np.poly1d(p_parabola)
plt.errorbar(x, y, yerr = dy, xerr = None, fmt = 'k.'); #same as before
plt.plot(x, f_parabola(x), 'r-') #now plot the parabola and the line.
plt.plot(x, f(x), 'g-') #same as before
plt.xlabel('Indep Variable')
plt.ylabel('Dep Variable')
plt.legend(['Parabola', 'Linear', 'Data']);
#Look at the plot closely: Note that this changes the fit! the fit doesn't try as hard to hit the points with very small uncertainty.

In [ ]:
#Let's try cubic now
polynomial_degree = 3
p_cubic = np.polyfit(x, y, polynomial_degree, w = 1/dy) ;  # now we include uncertainties again.
f_cubic = np.poly1d(p_cubic)
plt.errorbar(x, y, yerr = dy, xerr = None, fmt = 'k.'); #same as before
plt.plot(x, f_cubic(x), 'b-')
plt.plot(x, f_parabola(x), 'r-') #now plot the parabola and the line.
plt.plot(x, f(x), 'g-') #same as before
plt.xlabel('Indep Variable')
plt.ylabel('Dep Variable')
plt.legend(['Cubic', 'Parabola', 'Linear', 'Data']);
#Look at the plot closely: Note that this changes the fit! the fit doesn't try as hard to hit the points with very small uncertainty.

In [ ]:
#Since polyfit makes it's own chi-squared internally and hides it from us, if we want to see the actual chi-squared value resulting from the fit, we need to compute it ourselves.
#To make this easy for you in the future, I'll just define a new function to do this. If you're confused about this cell, don't worry, you'll see how to compute chi-squared below.
#Just make sure you evaluate this cell (or the equivalent one below in the summary) before trying to compute chi-squared yourself.

def chi2(data, unc, model): #define ("def") a function called chi2 that takes as input variables named data, unc, and model, in that order.
  #here "data" is  the data values, unc is the uncertainty in the data, and model is the function we fit with polyfit.
  #These three varaibles should be numpy arrays--that is, numpy lists of data. See an example below to see what I mean.
  N = 1.00*len(data) #Look at the length of the x array to see how many data points you have. Multiply it by "1.00" to convert it into a "float" data type. If you don't know what this is, it's not really important.
  result = np.sum((data-model)**2/unc**2)/N # make chi-squared: sum up the (data-model)^2/unc^2, then divide it by N
  return result #send back to the user the value of chi2


In [ ]:
#Once we've defined that function, we can use it to compute a few values of chi2 for our different values quite easily.

linear_chi2 = chi2(y, dy, f(x))
parab_chi2 = chi2(y, dy, f_parabola(x))
cubic_chi2 = chi2(y, dy, f_cubic(x))
print('The chi^2 value of the linear model is: ', linear_chi2)
print('The chi^2 value of the parabolic model is: ', parab_chi2)
print('The chi^2 value of the cubic model is: ', cubic_chi2)


In [ ]:
#We can also make plots of the residuals.
res_linear = y-f(x); #make the residuals for the linear fit.
plt.errorbar(x, res_linear, yerr = dy, xerr = None, fmt = 'ko'); #plot it, using the same error bars as before
plt.plot(x, 0*x, '--') #this code makes a dotted horizontal line at zero which is just there to guide the eye.
plt.xlabel('Indep Variable');
plt.ylabel('Residuals [data-model]');

In [ ]:
#Make the residuals plot for the quadratic model.
res_parab = y-f_parabola(x);
plt.errorbar(x, res_parab, yerr = dy, xerr = None, fmt = 'ko');
plt.plot(x, 0*x, '--')
plt.xlabel('Indep Variable')
plt.ylabel('Residuals [data-model]')

In [ ]:
#Do the same for the cubic data.
res_cubic = y-f_cubic(x);
plt.errorbar(x, res_cubic, yerr = dy, xerr = None, fmt = 'ko');
plt.plot(x, 0*x, '--')
plt.xlabel('Indep Variable');
plt.ylabel('Residuals [data-model]');

In [ ]:
polynomial_degree = 30
x_fine = np.linspace(np.min(x), np.max(x), 300)
p_overfit = np.polyfit(x, y, polynomial_degree, w = 1/dy) ;  # now we include uncertainties again.
f_overfit = np.poly1d(p_overfit)
plt.errorbar(x, y, yerr = dy, xerr = None, fmt = 'k.'); #same as before
plt.plot(x_fine, f_overfit(x_fine), 'b-')
plt.ylim([-2, 10])
plt.xlabel('Indep Variable');
plt.ylabel('Dep Variable');
plt.legend(['Overfit!', 'Data']);

In [ ]:
#Overfit residuals
res_overfit = y-f_overfit(x);
plt.errorbar(x, res_overfit, yerr = dy, xerr = None, fmt = 'ko');
plt.plot(x, 0*x, '--');
plt.xlabel('Indep Variable');
plt.ylabel('Residuals [data-model]');


In [ ]:
overfit_chi2 = chi2(y, dy, f_overfit(x))
print('The chi^2 value of the overfit model is: ', overfit_chi2)

In [ ]:
#SUMMARY FOR LAB
# if you want more explanation of what's going on here, see the cells above.

#Import stuff:
import numpy as np
import matplotlib.pyplot as plt


#Define the chi-squared function (don't change this chunk of code), see above for explanation.
def chi2(data, unc, model):
  N = 1.00*len(data)
  result = np.sum((data-model)**2/unc**2)/N
  return result



#input experimental data:

x = np.array([1, 2, 3, 4, 5, 6]); #edit this line, these are made-up
y = np.array([2.3, 4.1, 6.7, 8.3, 10.5, 13]) #edit this line, these are made-up
unc = np.array([0.2, 0.3, 0.1, 0.2, 0.3, 0.2]) #edit this line, these are made-up

# We will use linear fits in lab, so we set polynomial_degree = 1
polynomial_degree = 1 ;

#Do the fitting.
p = np.polyfit(x, y, polynomial_degree, w = 1/unc) ;  # make the fit, no need to edit
f = np.poly1d(p) # convert the fit coefficients into a function, no need to edit

#Print some of the results:
print('The fit slope is ', p[0]) # no need to edit these printout lines.
print('The fit intercept is ', p[1])
print('The chi-squared of this fit is ', chi2(y, unc, f(x)))

#Plot:
plt.rcParams.update({'font.size': 15})
plt.errorbar(x, y, yerr = unc, xerr = None, fmt = 'k.'); #plot the data points
plt.plot(x, f(x), 'g-') #plot the fit line
plt.xlabel('x [units?]') #change this to correspond to what you measured
plt.ylabel('y [units?]')  #change this to correspond to what you measured
plt.legend(['Fit', 'Data']); #edit as you need
